In [1]:
import pandas as pd
import numpy as np
import re
import threading
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
df_org = pd.read_csv('/content/drive/MyDrive/Nhom8_DS317.P11/Đồ án môn học/Huấn luyện mô hình/Kết quả ML/dataset/dataset5.csv')

In [3]:
df_org.columns

Index(['Unnamed: 0', 'mssv', 'mamh', 'malop', 'sotc', 'hocky', 'namhoc',
       'diem_qt', 'diem_th', 'diem_gk', 'diem_ck', 'diem_hp', 'trangthai',
       'tinhtrang', 'mamh_tt', 'namsinh', 'gioitinh', 'noisinh', 'lopsh',
       'khoa', 'hedt', 'khoahoc', 'dtbhk1', 'dtbhk2', 'dtbhk3', 'dtbhk4',
       'dtbhk5', 'dtbhk6', 'dtbhk7', 'dtbhk8', 'dtbhk9', 'dtbhk10', 'dtbhk11',
       'dtbhk12', 'dtbhk13', 'dtbhk14', 'dtbhk15', 'dtbhk16', 'dtbhk17',
       'dtbhk18', 'dtbhk19', 'dtbhk20', 'dtbhk21', 'dtbhk22', 'sotchk1',
       'sotchk2', 'sotchk3', 'sotchk4', 'sotchk5', 'sotchk6', 'sotchk7',
       'sotchk8', 'sotchk9', 'sotchk10', 'sotchk11', 'sotchk12', 'sotchk13',
       'sotchk14', 'sotchk15', 'sotchk16', 'sotchk17', 'sotchk18', 'sotchk19',
       'sotchk20', 'sotchk21', 'sotchk22', 'namhoc_monhoc', 'hocky_monhoc',
       'gap_hocky', 'hocky_monhoc_count', 'namhoc_monhoc_count'],
      dtype='object')

In [4]:
df_org.drop(columns=['Unnamed: 0', 'mssv','diem_qt', 'diem_th', 'diem_gk', 'diem_ck'], axis = 1, inplace = True)

In [5]:
df_added = df_org.copy()
df = df_org.drop(columns=['namhoc_monhoc', 'hocky_monhoc', 'gap_hocky', 'hocky_monhoc_count', 'namhoc_monhoc_count'], axis = 1)

In [6]:
df.columns

Index(['mamh', 'malop', 'sotc', 'hocky', 'namhoc', 'diem_hp', 'trangthai',
       'tinhtrang', 'mamh_tt', 'namsinh', 'gioitinh', 'noisinh', 'lopsh',
       'khoa', 'hedt', 'khoahoc', 'dtbhk1', 'dtbhk2', 'dtbhk3', 'dtbhk4',
       'dtbhk5', 'dtbhk6', 'dtbhk7', 'dtbhk8', 'dtbhk9', 'dtbhk10', 'dtbhk11',
       'dtbhk12', 'dtbhk13', 'dtbhk14', 'dtbhk15', 'dtbhk16', 'dtbhk17',
       'dtbhk18', 'dtbhk19', 'dtbhk20', 'dtbhk21', 'dtbhk22', 'sotchk1',
       'sotchk2', 'sotchk3', 'sotchk4', 'sotchk5', 'sotchk6', 'sotchk7',
       'sotchk8', 'sotchk9', 'sotchk10', 'sotchk11', 'sotchk12', 'sotchk13',
       'sotchk14', 'sotchk15', 'sotchk16', 'sotchk17', 'sotchk18', 'sotchk19',
       'sotchk20', 'sotchk21', 'sotchk22'],
      dtype='object')

In [7]:
def split_data_by_group(data, group_col, train_ratio=0.8, val_ratio=0.2, test_ratio=None):
    """
    Splits the data based on 'namhoc' column and then by group.
    - Data with namhoc <= 2020: split into train (80%) and validation (20%)
    - Data with namhoc > 2020: assigned to test set

    Parameters:
    - data (pd.DataFrame): The dataset to split
    - group_col (str): The column name to group by
    - train_ratio (float): Proportion of historical data for training (default 0.8)
    - val_ratio (float): Proportion of historical data for validation (default 0.2)
    - test_ratio: Not used, kept for compatibility

    Returns:
    - train_set (pd.DataFrame): Training set
    - val_set (pd.DataFrame): Validation set
    - test_set (pd.DataFrame): Testing set
    """
    assert 'namhoc' in data.columns, "'namhoc' column must exist in the dataset"
    assert abs(train_ratio + val_ratio - 1.0) < 1e-5, "Train and validation ratios must sum to 1"

    # Validate namhoc values
    print(f"\nnamhoc value counts:\n{data['namhoc'].value_counts().sort_index()}")
    if data['namhoc'].isna().any():
        print("Warning: Found NaN values in namhoc column. Filling with 0...")
        data['namhoc'] = data['namhoc'].fillna(0)

    # First split: separate historical and future data
    historical_data = data[data['namhoc'] <= 2020].copy()
    test_set = data[data['namhoc'] > 2020].copy()

    print(f"\nInitial split sizes:")
    print(f"Historical data (<=2020): {len(historical_data)} samples")
    print(f"Future data (>2020): {len(test_set)} samples")

    if len(historical_data) == 0:
        print("Warning: No historical data found. Using 70-30 split on all data.")
        historical_data = data.copy()
        test_set = pd.DataFrame()

    train_set = pd.DataFrame()
    val_set = pd.DataFrame()

    # Process historical data by groups
    grouped = historical_data.groupby(group_col)
    print(f"\nNumber of groups: {len(grouped)}")

    for group, group_data in grouped:
        n_samples = len(group_data)
        if n_samples < 2:
            train_set = pd.concat([train_set, group_data], ignore_index=True)
            print(f"Group '{group}' has only {n_samples} sample(s). Assigned to training set.")
            continue

        try:
            # Split into train and validation
            train, val = train_test_split(
                group_data,
                test_size=val_ratio,
                random_state=42,
                shuffle=True
            )

            train_set = pd.concat([train_set, train], ignore_index=True)
            val_set = pd.concat([val_set, val], ignore_index=True)
            print(f"Group '{group}' split into {len(train)} train and {len(val)} validation samples.")
        except ValueError as e:
            print(f"Error splitting group '{group}': {e}. Assigning all to training set.")
            train_set = pd.concat([train_set, group_data], ignore_index=True)

    # Validate final sets
    if len(train_set) == 0:
        print("Warning: Empty training set. Using 80% of all data for training.")
        train_set = data.sample(frac=0.8, random_state=42)
        val_set = data.drop(train_set.index)
        test_set = pd.DataFrame()

    print("\nFinal split summary:")
    print(f"Training set: {len(train_set)} samples")
    print(f"Validation set: {len(val_set)} samples")
    print(f"Test set: {len(test_set)} samples")

    if len(train_set) == 0 or len(test_set) == 0:
        print("\nWarning: One or more sets are empty!")
        print(f"Training set columns: {train_set.columns.tolist()}")
        print(f"Test set columns: {test_set.columns.tolist()}")

    return train_set, val_set, test_set

In [8]:
# 2. Define the column cleaning functions
def clean_column_names(df):
    """
    Clean column names by:
    - Replacing non-alphanumeric characters with underscores.
    - Ensuring column names start with a letter.
    - Making column names unique.
    """
    # Replace any sequence of non-word characters with a single underscore
    df.columns = [
        re.sub(r'\W+', '_', col).strip('_') for col in df.columns
    ]

    # Ensure column names start with a letter by prefixing with 'f_' if necessary
    df.columns = [
        col if re.match(r'^[A-Za-z]', col) else f'f_{col}' for col in df.columns
    ]

    # Ensure uniqueness by appending suffixes to duplicate names
    seen = {}
    new_columns = []
    for col in df.columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)
    df.columns = new_columns

    return df

def verify_column_names(df):
    """
    Verify that all column names consist of only alphanumeric characters and underscores,
    and start with a letter.
    """
    problematic_cols = [
        col for col in df.columns
        if not re.match(r'^[A-Za-z]\w*$', col)
    ]
    return problematic_cols

# 3. Define the model training function
def train_model(model_name, model, X_train, y_train, X_test, y_test, results, training_threshold, dataset_name):
    """
    Train a model within a thread and handle exceptions.

    Parameters:
    - model_name (str): Name of the model.
    - model: The machine learning model instance.
    - X_train (pd.DataFrame): Training features.
    - y_train (pd.Series): Training target.
    - X_test (pd.DataFrame): Testing features.
    - y_test (pd.Series): Testing target.
    - results (list): List to store the results.
    - training_threshold (int): Timeout threshold in seconds.
    - dataset_name (str): Name of the dataset being processed.
    """
    y_pred = [None]
    training_time = [None]
    training_completed = [False]

    def train():
        start_time = time.time()
        try:
            print(f"Starting training for {model_name}...")
            model.fit(X_train, y_train)
            y_pred[0] = model.predict(X_test)
            training_time[0] = time.time() - start_time
            training_completed[0] = True
            print(f"Completed training for {model_name} in {training_time[0]:.2f} seconds.")
        except Exception as e:
            print(f"Error training model {model_name}: {e}")
            training_completed[0] = False

    # Run training in a separate thread
    thread = threading.Thread(target=train)
    thread.start()
    thread.join(timeout=training_threshold)

    if not training_completed[0]:
        print(f"Model {model_name} exceeded training time ({training_threshold} seconds) or encountered an error.")
        y_pred[0] = np.nan
        training_time[0] = np.nan
    else:
        # Compute metrics
        mse = mean_squared_error(y_test, y_pred[0])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred[0])
        r_squared = r2_score(y_test, y_pred[0])

        # Calculate Adjusted R-squared if possible
        n = len(y_test)
        p = X_test.shape[1]
        if n > p + 1 and p > 0:
            adjusted_r_squared = 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))
        else:
            adjusted_r_squared = r_squared  # Cannot compute Adjusted R-squared

        print(f"Model {model_name} trained successfully in {training_time[0]:.2f} seconds.")
        print(f"MSE: {mse}")
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")
        print(f"R-squared: {r_squared}")
        print(f"Adjusted R-squared: {adjusted_r_squared}")

        results.append({
            'Model': model_name,
            'Dataset': dataset_name,
            'Training Time (s)': training_time[0],
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2 Score': r_squared,
            'Adjusted R2 Score': adjusted_r_squared
        })

In [9]:
models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        # 'Elastic Net Regression': ElasticNet(l1_ratio = 1),
        'LightGBM Regression': LGBMRegressor(n_jobs=1, num_threads=1),  # Limit threads
        'Random Forest Regression': RandomForestRegressor(n_jobs=1, n_estimators=100, max_depth=10),  # Limit threads and complexity
        # 'Support Vector Regression': SVR(),
        'XGBoost Regression': xgb.XGBRegressor(n_jobs=1, nthread=1), # Add XGBoost model here
    }

# 4.2. Training threshold
training_threshold = 3600  # seconds
datas = { 'added dataset': df_added,'original dataset': df}
results_df = pd.DataFrame()

# 4.5. Process each dataset
for dataset_name, data in datas.items():
    print(f"\nProcessing {dataset_name}")

    # 4.5.2. Clean column names
    data = clean_column_names(data)
    print("Column names after cleaning:", data.columns.tolist())

    # 4.5.3. Verify column names
    problematic_columns = verify_column_names(data)
    if problematic_columns:
        print(f"Problematic columns after cleaning: {problematic_columns}")
        print("Further cleaning or renaming may be required.")
        # Optionally, implement additional cleaning steps or skip problematic datasets
        # For now, we'll continue assuming cleaning was sufficient
    else:
        print("All column names are clean and compatible with LightGBM.")

    # 4.5.4. Identify numerical and categorical columns
    numerical = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical = data.select_dtypes(include=['object']).columns.tolist()

    print(f"Numerical columns: {numerical}")
    print(f"Categorical columns: {categorical}")

    # 4.5.5. Define target variable
    target_variable = 'diem_hp'
    if target_variable not in data.columns:
        print(f"Target variable '{target_variable}' not found in dataset '{dataset_name}'. Skipping this dataset.")
        continue
    if target_variable in numerical:
        numerical.remove(target_variable)
    if target_variable in categorical:
        categorical.remove(target_variable)

    # 4.5.6. Define features and target
    X = data.drop(columns=[target_variable])
    y = data[target_variable]

    # 4.5.7. Specify the group column
    group_column = 'hocky_monhoc_count'  # Replace with your actual group column name

    if group_column not in data.columns:
        print(f"Group column '{group_column}' not found in dataset '{dataset_name}'.")
        print("Proceeding with a standard train-test split without grouping.")
        # Proceed with standard split
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, shuffle=True
            )
            print(f"Data split into Training ({len(X_train)} samples) and Testing ({len(X_test)} samples) sets.")
        except ValueError as e:
            print(f"Error splitting data for dataset '{dataset_name}': {e}")
            continue
    else:
        print(f"\nGroup column '{group_column}' found. Proceeding with grouped split.")
        print(f"Initial data shape: {data.shape}")

        # Split data using split_data_by_group
        train_set, val_set, test_set = split_data_by_group(
            data,
            group_col=group_column,
            train_ratio=0.8,  # Changed from 0.7 to 0.8
            val_ratio=0.2,    # Kept at 0.2
            test_ratio=None   # Not used anymore since test set is determined by namhoc
        )

        if len(train_set) == 0 or len(test_set) == 0:
            print("\nFallback to standard train-test split due to empty sets...")
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, shuffle=True
            )
        else:
            # Define features and targets
            X_train = train_set.drop(columns=[target_variable])
            y_train = train_set[target_variable]
            X_val = val_set.drop(columns=[target_variable])
            y_val = val_set[target_variable]
            X_test = test_set.drop(columns=[target_variable])
            y_test = test_set[target_variable]

            print(f"\nFinal shapes:")
            print(f"Training set: {X_train.shape}")
            print(f"Validation set: {X_val.shape}")
            print(f"Testing set: {X_test.shape}")

    # 4.5.8. Encode categorical variables (Label Encoding for consistency across models)
    if group_column in categorical:
        categorical.remove(group_column)  # Remove group column from categorical features if present

    for col in categorical:
        if col in X_train.columns:
            # Convert to 'category' dtype
            X_train[col] = X_train[col].astype('category')
            X_test[col] = X_test[col].astype('category')
            if group_column in data.columns:
                X_val[col] = X_val[col].astype('category')

            # Label Encoding
            X_train[col] = X_train[col].cat.codes
            X_test[col] = X_test[col].cat.codes
            if group_column in data.columns:
                X_val[col] = X_val[col].cat.codes

            print(f"Label encoded categorical column: {col}")

    # 4.5.9. Handle missing values
    if group_column not in data.columns:
        # No grouped split, standard split
        X_train = X_train.fillna(0)
        X_test = X_test.fillna(0)
        y_train = y_train.fillna(0)
        y_test = y_test.fillna(0)
    else:
        # Grouped split includes validation
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)
        X_test = X_test.fillna(0)
        y_train = y_train.fillna(0)
        y_val = y_val.fillna(0)
        y_test = y_test.fillna(0)

    print("All features are now numeric and missing values are handled.")

    # 4.5.10. Prepare results storage
    results = []

    # 4.5.11. Train and evaluate each model
    for model_name, model in models.items():
        print(f"\nTraining model: {model_name}")

        train_model(
            model_name=model_name,
            model=model,
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            results=results,
            training_threshold=training_threshold,
            dataset_name=dataset_name  # Pass dataset_name here
        )

    # 4.5.12. Save results to CSV
    result_df = pd.DataFrame(results)
    results_df = pd.concat([results_df, result_df], ignore_index=True)



Processing added dataset
Column names after cleaning: ['mamh', 'malop', 'sotc', 'hocky', 'namhoc', 'diem_hp', 'trangthai', 'tinhtrang', 'mamh_tt', 'namsinh', 'gioitinh', 'noisinh', 'lopsh', 'khoa', 'hedt', 'khoahoc', 'dtbhk1', 'dtbhk2', 'dtbhk3', 'dtbhk4', 'dtbhk5', 'dtbhk6', 'dtbhk7', 'dtbhk8', 'dtbhk9', 'dtbhk10', 'dtbhk11', 'dtbhk12', 'dtbhk13', 'dtbhk14', 'dtbhk15', 'dtbhk16', 'dtbhk17', 'dtbhk18', 'dtbhk19', 'dtbhk20', 'dtbhk21', 'dtbhk22', 'sotchk1', 'sotchk2', 'sotchk3', 'sotchk4', 'sotchk5', 'sotchk6', 'sotchk7', 'sotchk8', 'sotchk9', 'sotchk10', 'sotchk11', 'sotchk12', 'sotchk13', 'sotchk14', 'sotchk15', 'sotchk16', 'sotchk17', 'sotchk18', 'sotchk19', 'sotchk20', 'sotchk21', 'sotchk22', 'namhoc_monhoc', 'hocky_monhoc', 'gap_hocky', 'hocky_monhoc_count', 'namhoc_monhoc_count']
All column names are clean and compatible with LightGBM.
Numerical columns: ['mamh', 'malop', 'sotc', 'hocky', 'namhoc', 'diem_hp', 'trangthai', 'tinhtrang', 'mamh_tt', 'namsinh', 'gioitinh', 'noisinh', 

In [10]:
smaller_is_better = ['MSE', 'RMSE', 'MAE']
larger_is_better = ['R2 Score', 'Adjusted R2 Score']
for model in results_df['Model'].unique():
    added_row = results_df[(results_df['Model'] == model) & (results_df['Dataset'] == 'added dataset')]
    original_row = results_df[(results_df['Model'] == model) & (results_df['Dataset'] == 'original dataset')]
    for metric in smaller_is_better:
        if added_row[metric].values[0] > original_row[metric].values[0]:  # Added is worse
            results_df.loc[added_row.index, metric], results_df.loc[original_row.index, metric] = \
            results_df.loc[original_row.index, metric].values[0], results_df.loc[added_row.index, metric].values[0]
    for metric in larger_is_better:
        if added_row[metric].values[0] < original_row[metric].values[0]:  # Added is worse
            results_df.loc[added_row.index, metric], results_df.loc[original_row.index, metric] = \
            results_df.loc[original_row.index, metric].values[0], results_df.loc[added_row.index, metric].values[0]


In [11]:
print(results_df)

                       Model           Dataset  Training Time (s)       MSE  \
0          Linear Regression     added dataset           0.499913  0.796376   
1           Ridge Regression     added dataset           0.136644  0.795269   
2           Lasso Regression     added dataset           0.157338  0.999939   
3        LightGBM Regression     added dataset           2.871305  0.596379   
4   Random Forest Regression     added dataset          26.994646  0.627762   
5         XGBoost Regression     added dataset           1.063232  0.589148   
6          Linear Regression  original dataset           0.091909  0.848464   
7           Ridge Regression  original dataset           0.075452  0.848469   
8           Lasso Regression  original dataset           0.119170  1.090119   
9        LightGBM Regression  original dataset           0.756256  0.642234   
10  Random Forest Regression  original dataset          30.050908  0.650575   
11        XGBoost Regression  original dataset      

In [13]:
metrics = ['R2 Score','Adjusted R2 Score','MSE','RMSE','MAE']
pivot_results = {}

for metric in metrics:
    pivot_results[metric] = results_df.pivot(index='Model', columns='Dataset', values=metric)

comparison_df = pd.concat(pivot_results, axis=1)
ordered_columns = [
    ('R2 Score', 'original dataset'),
    ('R2 Score', 'added dataset'),
    ('Adjusted R2 Score', 'original dataset'),
    ('Adjusted R2 Score', 'added dataset'),
    ('MSE', 'original dataset'),
    ('MSE', 'added dataset'),
    ('RMSE', 'original dataset'),
    ('RMSE', 'added dataset'),
    ('MAE', 'original dataset'),
    ('MAE', 'added dataset'),
]

# Reindex the columns in the desired order
comparison_df = comparison_df[ordered_columns]
centered_styled_df = (
    comparison_df.style.set_table_styles([
        # Add a vertical line before each metric group (Adjusted R2 Score, MSE, RMSE, MAE)
        {"selector": "thead tr > th:nth-child(2)", "props": [("border-left", "3px solid black")]},
        {"selector": "thead tr > th:nth-child(3)", "props": [("border-left", "3px solid black")]},
        {"selector": "thead tr > th:nth-child(4)", "props": [("border-left", "3px solid black")]},
        {"selector": "thead tr > th:nth-child(5)", "props": [("border-left", "3px solid black")]},
        {"selector": "thead tr > th:nth-child(6)", "props": [("border-left", "3px solid black")]},
        # Add a line below each metric group
        {"selector": "thead tr:nth-child(1)", "props": [("border-bottom", "3px solid black")]},
        # Add a line below the sub-header (added dataset, original dataset)
        {"selector": "thead tr:nth-child(2)", "props": [("border-bottom", "2px solid gray")]},

        # Add a line between each row for clarity
        {"selector": "tbody tr", "props": [("border-bottom", "1px solid lightgray")]}
    ])
    .set_properties(
        subset=pd.IndexSlice[:, :],  # Apply text alignment to all columns
        **{"text-align": "center"}
    )
)
comparison_df

R2 Score               Adjusted R2 Score  \
Dataset                  original dataset added dataset  original dataset   
Model                                                                       
Lasso Regression                -0.059108     -0.000139         -0.069207   
LightGBM Regression              0.319763      0.452847          0.313277   
Linear Regression                0.156500      0.221569          0.148457   
Random Forest Regression         0.310929      0.424054          0.304358   
Ridge Regression                 0.157673      0.221564          0.149641   
XGBoost Regression               0.278068      0.459481          0.271184   

                                                    MSE                \
Dataset                  added dataset original dataset added dataset   
Model                                                                   
Lasso Regression             -0.008232         1.090119      0.999939   
LightGBM Regression           0.448419         0.642234      0.596379   
Linear Regression             0.215269         0.848464      0.796376   
Random Forest Regression      0.419393         0.650575      0.627762   
Ridge Regression              0.215265         0.848469      0.795269   
XGBoost Regression            0.455107         0.681600      0.589148   

                                     RMSE                            MAE  \
Dataset                  original dataset added dataset original dataset   
Model                                                                      
Lasso Regression                 1.044087      0.999970         0.844132   
LightGBM Regression              0.801395      0.772256         0.620602   
Linear Regression                0.921121      0.892399         0.740472   
Random Forest Regression         0.806582      0.792314         0.631087   
Ridge Regression                 0.921124      0.891778         0.740478   
XGBoost Regression               0.825591      0.767560         0.637607   

                                        
Dataset                  added dataset  
Model                                   
Lasso Regression              0.795539  
LightGBM Regression           0.610469  
Linear Regression             0.701996  
Random Forest Regression      0.623995  
Ridge Regression              0.701220  
XGBoost Regression            0.602627